<header> <h2><center>Present notebook is an exercize for IBM Data Science course on Coursera.</center> </h2></header>
    
   <b> The scope of the exercize is to grab the list of Toronto neighbourhoods from Wikipedia page, then add coordinates to each of them using the Geocoder library and then cluster the neighborhoods based on their most popular venues using Foursquare data. <b>

<header><h4> First part </h4></header>
  <p> <strong> 1. Start with importing the libraries<strong>

In [15]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

<strong>2. Then parsing a table from the web page<strong>

In [16]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' #URL of a page containing desired table
html_doc = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M") #parsing the entire page html code
html_doc='\"\"\"'+html_doc.text+'\"\"\"' #adding the quotas necessary for BeautifulSoup
soup = BeautifulSoup(html_doc,'lxml') #creating a BeautifulSoup object with html document in it
table = soup.find_all('table')[0] #cutting out the table from the html code of entire page
table.text[:100] # checking the table code

'\n\nPostal Code\n\nBorough\n\nNeighborhood\n\n\nM1A\n\nNot assigned\n\n\n\n\nM2A\n\nNot assigned\n\n\n\n\nM3A\n\nNorth York\n\n'

In [17]:
#defining the function which will create a list from the html string of a table
def tableDataText(table):       
    rows = []
    trs = table.find_all('tr')
    headerow = [td.get_text(strip=True) for td in trs[0].find_all('th')] # header row
    if headerow: # if there is a header row include first
        rows.append(headerow)
        trs = trs[1:]
    for tr in trs: # for every table row
        rows.append([td.get_text(strip=True) for td in tr.find_all('td')]) # data row
    return rows

<strong>3. Converting the table to a Pandas Dataframe and cleaning it to a desired look<strong>

In [18]:
list_table = tableDataText(table) #getting a table from an html string using the function
df=pd.DataFrame(list_table) #converting a list to a Pandas dataframe
df.columns = df.iloc[0] #replacing the header with a top row and reindexing the dataframe
df = df.reindex(df.index.drop(0)).reset_index(drop=True) 
df.columns.name = None
df.drop(df[df.Borough == 'Not assigned'].index, inplace=True)#dropping the voids
df.reset_index(drop=True, inplace=True)
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [19]:
df.shape #checking the shape of a dataframe

(103, 3)

<header><h4> Second part </h4></header>
  <p> <strong>1. Adding the location data to a dataframe using Geocoder<strong>

In [21]:
!conda install -c conda-forge geocoder #instaling the Geocoder package

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          59 KB

The following NEW packages will be INSTALLED:

    geocoder: 1.38.1-py_1 conda-forge
    ratelim:  0.1.6-py_2  conda-forge


ratelim-0.1.6        | 6 KB      | ##################################### | 100% 
geocoder-1.38.1      | 53 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [23]:
import geocoder

In [24]:
latitude=[] #creating a dummy lists for coordinates
longitude=[]
for i in range(0,103): #looping through the rows of dataframe, obtaining the latitude and longitude for each postal code and adding it to a lists
 latitude.append(geocoder.arcgis('{},Toronto, Ontario'.format(df['Postal Code'][i])).latlng[0])
 longitude.append(geocoder.arcgis('{},Toronto, Ontario'.format(df['Postal Code'][i])).latlng[1])
df['Latitude']=latitude #adding the coloumns with coordinates to a dataframe
df['Longitude']=longitude
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752935,-79.335641
1,M4A,North York,Victoria Village,43.728102,-79.311890
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.723265,-79.451211
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390


<header><h4> Third part </h4></header>
  <p> <strong>1. Plotting the neighborhoods

In [4]:
#importing the libraries
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans #importing the clustering model

!conda install -c conda-forge folium=0.5.0 --yes # installing the map rendering library
import folium 

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    ------------------------------------------------------------
                       

In [25]:
from geopy.geocoders import Nominatim
#getting the ccordinates of Toronto
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [26]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10) #creating a map centered on Toronto coordinates

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [41]:
from IPython.core.display import HTML
HTML(map_toronto._repr_html_())

<strong> 2. Getting the venue data for each neighborhood using Foursquare API

In [27]:
#Defining the keys for Foursquare API
CLIENT_ID = 'JMZZWZQPHQD3T0LH5TSVNIUGPRCTOZCDPUCJMK5YGALS2MXZ' #Foursquare ID
CLIENT_SECRET = 'WR0KHTDPTLZIJOSHZVP5TFAMVEVJVD2LCLZFU0FTRQHRHS4P' #Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [28]:
#defining the function which will return the nearby venues for neighbourhood
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    LIMIT=100
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [29]:
#passing the Toronto neighborhoods data to a function and assigning the resultant venue list to a dataframe
toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude'],
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
Bayview Village
Downsview
The Danforth West, Ri

In [30]:
#shape of the resultant venue dataframe
print(toronto_venues.shape)
toronto_venues.head()

(2253, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.752935,-79.335641,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.752935,-79.335641,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Parkwoods,43.752935,-79.335641,Sun Life,43.754760,-79.332783,Construction & Landscaping
3,Parkwoods,43.752935,-79.335641,MacLeod Exteriors Inc.,43.755014,-79.338688,Construction & Landscaping
4,Victoria Village,43.728102,-79.311890,Tim Hortons,43.725517,-79.313103,Coffee Shop


<strong>3. Clustering the neighborhoods based on the venue data

In [31]:
#preparing the dataframe with types of venues in each neighborhood
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="") #creating a dataframe with coloumn for each type of venue
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] # add neighborhood column to dataframe
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1]) # move neighborhood column to the first column
toronto_onehot = toronto_onehot[fixed_columns]
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index() #grouping the rows by neighborhood to get the mean number of each type of venue in each neighborhood

In [32]:
#defining the function which will sort the venues based on their popularity
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [33]:
#Creating a dataframe with top venues in each neighborhood
num_top_venues = 10 #number of top venues we look at for each neighborhood

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Breakfast Spot,Skating Rink,Badminton Court,Sushi Restaurant,Supermarket,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Doctor's Office
1,"Alderwood, Long Branch",Pizza Place,Gym,Sandwich Place,Pub,Coffee Shop,Convenience Store,Gas Station,Pharmacy,Ethiopian Restaurant,Falafel Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Sandwich Place,Sushi Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Restaurant,Diner,Bridal Shop,Gas Station
3,Bayview Village,Trail,Construction & Landscaping,Women's Store,Donut Shop,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Falafel Restaurant,Farm
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Sandwich Place,Coffee Shop,Sushi Restaurant,Restaurant,Café,Butcher,Sports Club,Pub,Thai Restaurant


In [34]:
# vlustering the neighborhoods based on the venue data
kclusters = 5 # set number of clusters

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering) # creating a k-means cluster object and fitting it with venue data
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_) # adding the cluster number to the venue dataframe 

toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.752935,-79.335641,0.0,Construction & Landscaping,Food & Drink Shop,Park,Women's Store,Falafel Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant
1,M4A,North York,Victoria Village,43.728102,-79.311890,4.0,Pizza Place,Coffee Shop,Intersection,French Restaurant,Park,Portuguese Restaurant,Farm,Falafel Restaurant,Ethiopian Restaurant,Discount Store
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041,4.0,Pub,Café,Athletics & Sports,Coffee Shop,Performing Arts Venue,Tech Startup,Seafood Restaurant,Mediterranean Restaurant,Boutique,Mexican Restaurant
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.723265,-79.451211,4.0,Clothing Store,Food Court,Cosmetics Shop,Pharmacy,American Restaurant,Bookstore,Toy / Game Store,Men's Store,Restaurant,Furniture / Home Store
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390,4.0,Coffee Shop,Café,Sushi Restaurant,Hobby Shop,Fried Chicken Joint,Sandwich Place,Pharmacy,Park,Middle Eastern Restaurant,Italian Restaurant


In [35]:
# creating a resultant dataframe containing all the data for each neighborhood
toronto_merged = toronto_merged.fillna(0)
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype('int32')
toronto_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.752935,-79.335641,0,Construction & Landscaping,Food & Drink Shop,Park,Women's Store,Falafel Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant
1,M4A,North York,Victoria Village,43.728102,-79.311890,4,Pizza Place,Coffee Shop,Intersection,French Restaurant,Park,Portuguese Restaurant,Farm,Falafel Restaurant,Ethiopian Restaurant,Discount Store
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041,4,Pub,Café,Athletics & Sports,Coffee Shop,Performing Arts Venue,Tech Startup,Seafood Restaurant,Mediterranean Restaurant,Boutique,Mexican Restaurant
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.723265,-79.451211,4,Clothing Store,Food Court,Cosmetics Shop,Pharmacy,American Restaurant,Bookstore,Toy / Game Store,Men's Store,Restaurant,Furniture / Home Store
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390,4,Coffee Shop,Café,Sushi Restaurant,Hobby Shop,Fried Chicken Joint,Sandwich Place,Pharmacy,Park,Middle Eastern Restaurant,Italian Restaurant


<strong> 4. Plotting the data on a map


In [40]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11) #creating a map centered on Toronto

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [42]:
HTML(map_clusters._repr_html_())